# Decision Tree Regressor with Pyspark's Structured API

Importing necessary packages

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, udf
from math import radians, sin, cos, sqrt, atan2
import itertools
from pyspark.sql.types import DoubleType

## 1. Preparation

Let's initialize a Spark session:

In [2]:
#pyspark init
builder = SparkSession.Builder().appName('taxi_duration_highAPI')
spark = builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/11 20:05:07 WARN Utils: Your hostname, HP-Envy resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/11 20:05:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/11 20:05:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Read input datasets for training and testing:

In [3]:
#Read input files
raw_train_data = spark.read.csv('train.csv', header=True, inferSchema=True)
raw_test_data = spark.read.csv('test.csv', header=True, inferSchema=True)

## 2. Data preprocessing

Parse timestamp features in the dataset:

In [4]:
#Cast the pickup string values of training data into timestamps.
casted_train_data = raw_train_data.withColumns({
                        'pickup_datetime' : raw_train_data['pickup_datetime'].cast('timestamp'),
                    })

#Cast the pickup string values of testing data into timestamps.
casted_test_data = raw_test_data.withColumns({
                        'pickup_datetime' : raw_test_data['pickup_datetime'].cast('timestamp'),
                    })

User-Defined Function (UDF)

**Tính Khoảng Cách Haversine**

Hàm `haversine` được sử dụng để tính **khoảng cách địa lý** giữa hai điểm trên bề mặt Trái Đất dựa trên tọa độ kinh độ (longitude) và vĩ độ (latitude). Thêm đặc trưng này cho bài toán dự đoán thời gian chuyến đi taxi (`trip_duration`), vì thời gian thường phụ thuộc vào quãng đường thực tế giữa điểm đón và điểm trả khách.

**Công thức Haversine**

Công thức Haversine tính khoảng cách giữa hai điểm trên một hình cầu (giả định Trái Đất là hình cầu với bán kính trung bình $ R = 6371 \, \text{km} $):

$$
a = \sin^2\left(\frac{\Delta \text{lat}}{2}\right) + \cos(\text{lat}_1) \cdot \cos(\text{lat}_2) \cdot \sin^2\left(\frac{\Delta \text{lon}}{2}\right)
$$ 

$$
c = 2 \cdot \text{atan2}\left(\sqrt{a}, \sqrt{1-a}\right)
$$

$$
d = R \cdot c
$$

Trong đó:
- $ \text{lon}_1, \text{lat}_1 $: Kinh độ và vĩ độ của điểm đầu tiên (điểm đón khách).
- $ \text{lon}_2, \text{lat}_2 $: Kinh độ và vĩ độ của điểm thứ hai (điểm trả khách).
- $ \Delta \text{lon} = \text{lon}_2 - \text{lon}_1 $: Chênh lệch kinh độ.
- $ \Delta \text{lat} = \text{lat}_2 - \text{lat}_1 $: Chênh lệch vĩ độ.
- $ R = 6371 \, \text{km} $: Bán kính Trái Đất.
- $ d $: Khoảng cách giữa hai điểm (km).
- $ a $: Biểu diễn "haversine" của góc giữa hai điểm.
- $ c $: Tính góc trung tâm (radian) giữa hai điểm trên hình cầu.
- $ d $: Nhân góc trung tâm với bán kính Trái Đất để ra khoảng cách (km)

In [5]:
# Hàm tính khoảng cách Haversine
def haversine(lon1, lat1, lon2, lat2):
    if None in (lon1, lat1, lon2, lat2) or not all(isinstance(x, (int, float)) for x in [lon1, lat1, lon2, lat2]):
        return 0.0
    R = 6371  # Bán kính Trái Đất (km)
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

haversine_udf = udf(haversine, DoubleType())

Extract usable features:

In [6]:
#Get usable columns from the dataframe
#Also convert timestamps into time elements and encode `store_and_fwd_flag` feature into binary values
extracted_train_df = casted_train_data.selectExpr(
    'id',
    'vendor_id',
    'YEAR(pickup_datetime) AS pickup_year',
    'MONTH(pickup_datetime) AS pickup_month',
    'DAY(pickup_datetime) AS pickup_day',
    'HOUR(pickup_datetime) AS pickup_hour',
    'MINUTE(pickup_datetime) AS pickup_min',
    'SECOND(pickup_datetime) AS pickup_sec',
    'passenger_count',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'CASE WHEN store_and_fwd_flag = "Y" THEN 1 ELSE 0 END AS store_and_fwd_flag',
    'trip_duration'
).withColumn('distance_km', haversine_udf(col('pickup_longitude'), col('pickup_latitude'), col('dropoff_longitude'), col('dropoff_latitude')))

extracted_test_df = casted_test_data.selectExpr(
    'id',
    'vendor_id',
    'YEAR(pickup_datetime) AS pickup_year',
    'MONTH(pickup_datetime) AS pickup_month',
    'DAY(pickup_datetime) AS pickup_day',
    'HOUR(pickup_datetime) AS pickup_hour',
    'MINUTE(pickup_datetime) AS pickup_min',
    'SECOND(pickup_datetime) AS pickup_sec',
    'passenger_count',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'CASE WHEN store_and_fwd_flag = "Y" THEN 1 ELSE 0 END AS store_and_fwd_flag'
).withColumn('distance_km', haversine_udf(col('pickup_longitude'), col('pickup_latitude'), col('dropoff_longitude'), col('dropoff_latitude')))

In [7]:
# Loại bỏ outliers
extracted_train_df = extracted_train_df.filter(
    (col('trip_duration') > 0) & 
    (col('trip_duration') < 36000) &  # Giới hạn 10 giờ
    (col('passenger_count') >= 1) & 
    (col('passenger_count') <= 6) &  # Giới hạn hành khách
    (col('distance_km') > 0) & 
    (col('distance_km') < 100)  # Giới hạn khoảng cách 100km
)

# Kiểm tra giá trị null
print("Kiểm tra giá trị null trong extracted_train_df:")
for column in extracted_train_df.columns:
    null_count = extracted_train_df.filter(col(column).isNull()).count()
    print(f"{column}: {null_count} giá trị null")

extracted_train_df = extracted_train_df.na.drop()

Kiểm tra giá trị null trong extracted_train_df:


id: 0 giá trị null


vendor_id: 0 giá trị null


pickup_year: 0 giá trị null


pickup_month: 0 giá trị null


...

pickup_latitude: 0 giá trị null


dropoff_longitude: 0 giá trị null


dropoff_latitude: 0 giá trị null
store_and_fwd_flag: 0 giá trị null


trip_duration: 0 giá trị null


distance_km: 0 giá trị null


Encode the features into vector of values:

In [8]:
#Extract training features name
# train_cols = [col for col in extracted_train_df.columns if col != 'trip_duration']
# test_cols = extracted_test_df.columns
# Danh sách các cột đặc trưng (loại bỏ 'id' và 'trip_duration')
feature_cols = ['vendor_id', 'pickup_year', 'pickup_month', 'pickup_day', 'pickup_hour',
                'pickup_min', 'pickup_sec', 'passenger_count', 'pickup_longitude',
                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 
                'store_and_fwd_flag', 'distance_km']

#Use VectorAssembler to transform feature columns into a single vector
# assember_train = VectorAssembler(inputCols=train_cols, outputCol='features')
# assember_test = VectorAssembler(inputCols=test_cols, outputCol='features')
extracted_train_df.printSchema()
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

# train_data = assember_train.transform(extracted_train_df).select('features', 'trip_duration')
# test_data = assember_test.transform(extracted_test_df).select('features')
# Chuyển đổi dữ liệu train
train_data = assembler.transform(extracted_train_df).select('id', 'features', 'trip_duration')

root
 |-- id: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_year: integer (nullable = true)
 |-- pickup_month: integer (nullable = true)
 |-- pickup_day: integer (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- pickup_min: integer (nullable = true)
 |-- pickup_sec: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- store_and_fwd_flag: integer (nullable = false)
 |-- trip_duration: integer (nullable = true)
 |-- distance_km: double (nullable = true)



## 3. Model training

Split data

In [9]:
#Split data into training and testing sets

#Training set proportion parameter:
train_size = 0.8

train, validation = train_data.randomSplit([train_size, 1 - train_size], seed=24) #Fixed with seed for reproductivity

Fine-tuning

In [10]:
# Fine-tuning
param_grid = {
    'maxDepth': [5, 10],
    'maxBins': [32, 64],
    'minInstancesPerNode': [1, 5]
}

best_rmse = float('inf')
best_params = None
best_model = None

for maxDepth, maxBins, minInstancesPerNode in itertools.product(param_grid['maxDepth'], param_grid['maxBins'], param_grid['minInstancesPerNode']):
    dt = DecisionTreeRegressor(
        featuresCol='features',
        labelCol='trip_duration',
        maxDepth=maxDepth,
        maxBins=maxBins,
        minInstancesPerNode=minInstancesPerNode
    )
    model = dt.fit(train)
    predictions = model.transform(validation)
    evaluator_rmse = RegressionEvaluator(labelCol='trip_duration', predictionCol='prediction', metricName='rmse')
    rmse = evaluator_rmse.evaluate(predictions)
    print(f"Params: maxDepth={maxDepth}, maxBins={maxBins}, minInstancesPerNode={minInstancesPerNode}, RMSE={rmse}")
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_params = {'maxDepth': maxDepth, 'maxBins': maxBins, 'minInstancesPerNode': minInstancesPerNode}
        best_model = model

Params: maxDepth=5, maxBins=32, minInstancesPerNode=1, RMSE=412.3085799814713


Params: maxDepth=5, maxBins=32, minInstancesPerNode=5, RMSE=412.3085799814713


Params: maxDepth=5, maxBins=64, minInstancesPerNode=1, RMSE=412.5743608321677


Params: maxDepth=5, maxBins=64, minInstancesPerNode=5, RMSE=412.5743608321677


Params: maxDepth=10, maxBins=32, minInstancesPerNode=1, RMSE=385.1342661800246


Params: maxDepth=10, maxBins=32, minInstancesPerNode=5, RMSE=383.84146700203235


Params: maxDepth=10, maxBins=64, minInstancesPerNode=1, RMSE=387.4160455173831


Params: maxDepth=10, maxBins=64, minInstancesPerNode=5, RMSE=382.92100639699294


## 4. Model evaluation (hold-out)

In [11]:
# Đánh giá mô hình tốt nhất
predictions = best_model.transform(validation)
evaluator_rmse = RegressionEvaluator(labelCol='trip_duration', predictionCol='prediction', metricName='rmse')
evaluator_r2 = RegressionEvaluator(labelCol='trip_duration', predictionCol='prediction', metricName='r2')
rmse = evaluator_rmse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)

print(f"Best Params: {best_params}")
print(f"Best RMSE trên validation: {rmse}")
print(f"Best R2 trên validation: {r2}")
print("Feature Importances:", best_model.featureImportances)
print("Cấu trúc cây quyết định:")
print(best_model.toDebugString)

Best Params: {'maxDepth': 10, 'maxBins': 64, 'minInstancesPerNode': 5}
Best RMSE trên validation: 382.92100639699294
Best R2 trên validation: 0.68297963107283
Feature Importances: (14,[0,2,3,4,5,6,7,8,9,10,11,13],[3.8945903606860175e-05,0.00551565866897974,0.0004325258737785563,0.0820791590910963,0.0007115081196333648,0.00025764296340473954,0.00015758419861649256,0.012786282155372374,0.0037506737937210125,0.008494182092937057,0.022048028110449522,0.8637278090284042])
Cấu trúc cây quyết định:
DecisionTreeRegressionModel: uid=DecisionTreeRegressor_0ab1456ed698, depth=10, numNodes=1991, numFeatures=14
  If (feature 13 <= 4.424373410690684)
   If (feature 13 <= 1.9387036111214586)
    If (feature 13 <= 1.1999678748805256)
     If (feature 13 <= 0.9049156566656722)
      If (feature 11 <= 40.768659591674805)
       If (feature 4 <= 7.5)
        If (feature 13 <= 0.7547940517197789)
         If (feature 4 <= 6.5)
          If (feature 4 <= 4.5)
           If (feature 10 <= -73.98379135131836

## 5. Prediction (test file)

In [12]:
# Dự đoán trên tập test
test_data = assembler.transform(extracted_test_df).select('id', 'features')
test_predictions = model.transform(test_data).select('id', 'prediction').withColumnRenamed('prediction', 'trip_duration')

# Write file
test_predictions.coalesce(1).write.csv('prediction_highAPI.csv', header=True, mode='overwrite')

# Hiển thị một số dòng của prediction
test_predictions.show(5)

# Đóng SparkSession
spark.stop()

25/04/11 20:09:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+------------------+
|       id|     trip_duration|
+---------+------------------+
|id3004672| 819.1158240155567|
|id3505355| 683.2462006079028|
|id1217141|456.87959454338767|
|id2150126|1297.9835647853229|
|id1598245| 366.8636084122494|
+---------+------------------+
only showing top 5 rows

